# Соревнование на Kaggle

                                                                                                        Марк и Лия. БКЛ-151.

### Загрузка данных конкурса

In [1]:
import subprocess
import os

DOWNLOAD_ARGS = [
    "kaggle", "competitions", "download", "-c", "avito-demand-prediction",
    "-f", "train.csv.zip","-p", "data/"
]
SUBMIT_ARGS = [
    "kaggle", "competitions", "submit", "-c", "avito-demand-prediction", "-f",
    "submission.csv"
]

In [2]:
downloader_process = subprocess.Popen(
    DOWNLOAD_ARGS, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if not os.path.exists("data/train.csv.zip"):
    downloader_process.communicate()

### Распаковка и импортирование

In [3]:
import pandas as pd
import numpy as np
import zipfile
import os
import re

In [4]:
if not os.path.exists("data/train.csv"):
    zip_train = zipfile.ZipFile("data/train.csv.zip", 'r')
    zip_train.extractall(path='data/')
    zip_train.close()

Загружаем одну 50ую от тренировочного файла, из-за недостаточности ресурсов 

In [5]:
train_df = pd.read_csv('data/train.csv')
train_df = train_df[:len(train_df)//50]
train_df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


Проверяем размерность

In [6]:
train_df.shape

(30068, 18)

### Подготовка данных

In [7]:
import sklearn.model_selection
import sklearn.metrics
import sklearn.linear_model
import sklearn.feature_extraction
import lightgbm as lgb
import nltk
import scipy as sp
import texterra
import ner
import pymorphy2
import ufal_mod

Инициализируем несколько моделей: модель лемматизатора (pymorphy2), модель для извлечения именованых сущностей и загружаем список стоп слов для русского языка

In [47]:
%%capture
t = texterra.API()
syntax_model = ufal_mod.Model('russian-syntagrus-ud-2.0-170801.udpipe')
morph = pymorphy2.MorphAnalyzer()
extractor = ner.Extractor()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru');
russian_stops = nltk.corpus.stopwords.words('russian')

INFO:tensorflow:Restoring parameters from D:\Anaconda3\envs\labs_ml\lib\site-packages\ner\extractor\..\model\ner_model


Программируем функционал оценивания RMSE, для будущей оценки моделей

In [9]:
def rmse(predictions, targets):
    differences = predictions - targets                       
    differences_squared = differences ** 2                   
    mean_of_differences_squared = differences_squared.mean()  
    rmse_val = np.sqrt(mean_of_differences_squared)         
    return rmse_val                                          

Переводим функционал в оценщик моделей

In [10]:
rmse_scorer = sklearn.metrics.make_scorer(rmse)

Напишем функцию для очистки ячеек от нерусского языка, и некоторых знаков препинания; это нужно для некоторых из дальнейших морфологических обрабатывающих функционалов

In [11]:
def clear_text(text):
    clean_text = re.sub(u"[^а-яА-Я0-9.,\-\s]", " ", text)
    clean_text = re.sub(u"[.,\-\s]{3,}", " ", clean_text)
    return clean_text

Программируем функционал для деления на выборки (хотя в дальнейшем оказалось что это не требуется)

In [12]:
def get_train_test_split(X, y):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y, train_size=0.8)
    return X_train, y_train, X_test, y_test

Выбранная модель регрессора -- LGBM, в основном за скорость. Параметры было решено не отбирать, а оценка делалась на кросс валидации (не хотелось заниматься играми с моделями, потому что это не совсем про NLP)

In [13]:
def get_score_lgbm(X, y):
    estimator = lgb.LGBMRegressor(device='gpu', task='train',
                                 boosting_type='gbdt', metric = 'rmse',
                                 verbose=0)

    score = np.mean(sklearn.model_selection.cross_val_score(estimator, X, y, scoring= rmse_scorer, cv=5))
    return score

Выделяем выборку описаний, и целевой вектор, заменяем NA на пустые строчки, и очищаем от мусора описания

In [14]:
X = train_df['description']
y = train_df['deal_probability']
X = X.fillna('')
X = X.apply(clear_text)

Первый векторизатор -- самый элементарный подсчитывающий, без каких-то дополнений; однако стоп слова было решено убирать сразу 

In [15]:
vec_1gram = sklearn.feature_extraction.text.CountVectorizer(
    stop_words=russian_stops, dtype=np.float64)
X_1gram = vec_1gram.fit_transform(X)

In [16]:
%%capture
score_1gram = get_score_lgbm(X_1gram, y)

Простой векторизатор получает оценку RMSE на тестировании 0.24759

Следующий векторизатор -- это TF-IDF

In [15]:
vec_1gram_tfidf = sklearn.feature_extraction.text.TfidfVectorizer(
    stop_words=russian_stops)
X_1gram_tfidf = vec_1gram_tfidf.fit_transform(X)

In [16]:
%%capture
tfidf_1gram_score = get_score_lgbm(X_1gram_tfidf, y)

TF-IDF векторизатор улучшает оценку на  0.00011

Следующий это 3граммы, который мы планируем присоединить к TFIDF 1 граммам

In [17]:
vec_3gram_tfidf = sklearn.feature_extraction.text.TfidfVectorizer(
    lowercase=True,
    sublinear_tf=True,
    ngram_range=(3, 3))
X_3gram_tfidf = vec_3gram_tfidf.fit_transform(X)

In [18]:
X_13gram_tfidf = sp.sparse.hstack([X_1gram_tfidf, X_3gram_tfidf])

In [19]:
%%capture
score_13gram_tfidf = get_score_lgbm(X_13gram_tfidf, y)

3-граммы + 1-граммы TF-IDF улучшили результат по сравнению с TF-IDF 1-граммой на  0.00002

Делаем функцию, создающую последовательность POS тагов

In [20]:
def pos_sent_nltk(text):
    tags = nltk.pos_tag(nltk.word_tokenize(text), lang='rus')
    tag_sent = []
    for word in tags:
        tag_sent.append(word[1])
    tag_sent = " ".join(tag_sent)
    return tag_sent

In [21]:
X_pos_nltk = X.apply(pos_sent_nltk)

Векторизатор для POS был выбран обычный TFIDF монограммы

In [22]:
vec_pos_nltk = sklearn.feature_extraction.text.TfidfVectorizer(
    sublinear_tf=True)
X_pos_tfidf = vec_pos_nltk.fit_transform(X_pos_nltk)

In [23]:
X_13gram_pos_tfidf = sp.sparse.hstack([X_13gram_tfidf, X_pos_tfidf])

In [24]:
score_pos_13gram_tfidf = get_score_lgbm(X_13gram_pos_tfidf, y)

Добавление POS улучшило результат по сравнению с TF-IDF 1-граммами + 3-граммами на 0.00016

Функция выделяющая именованые сущности по типу POS тэгов, то есть последовательности

In [41]:
def determ_entities(sentence):
    try:
        entities = [ent.type for ent in extractor(sentence)]
    except ValueError:
        return ' '
    return ' '.join(entities)

In [42]:
X_ent = X.apply(determ_entities)

In [43]:
vec_ner = sklearn.feature_extraction.text.TfidfVectorizer(
    sublinear_tf=True)
X_ner_tfidf = vec_pos_nltk.fit_transform(X_ent)

In [49]:
X_plus_ent = sp.sparse.hstack([X_ner_tfidf, X_13gram_pos_tfidf])

In [66]:
score_plus_ent = get_score_lgbm(X_plus_ent, y)

Добавление NER ухудшило результат по сравнению с TF-IDF 1-граммами + 3-граммами + POS на 0.00011

Функция лемматизатор

In [59]:
def extract_lemma(sentence):
    lemmas = [morph.parse(word)[0].normal_form for word in nltk.word_tokenize(sentence)]
    return ' '.join(lemmas)

In [77]:
X_lemma = X.apply(extract_lemma)

In [78]:
vec_lemma = sklearn.feature_extraction.text.TfidfVectorizer(
    sublinear_tf=True)
X_lemma_tfidf = vec_lemma.fit_transform(X_lemma)

In [75]:
X_plus_lemma = sp.sparse.hstack([X_lemma_tfidf, X_13gram_pos_tfidf])

In [76]:
score_lemma = get_score_lgbm(X_plus_lemma, y)

Добавление лемматизации ухудшило результат по сравнению с TF-IDF 1-граммами + 3-граммами + POS на 0.00062

Также популярным способом поднять немного сгенерировать новых признаков из этих описаний

In [25]:
def len_of_sentence(sent):
    if sent != ' ':
        sent_len = len(sent)
    else:
        sent_len = 0
    return sent_len

In [26]:
def voskl(sent):
    voskl = re.compile(r"!")
    voskl_count = len(voskl.findall(sent))
    return voskl_count

In [27]:
def quest(sent):
    quest = re.compile(r"\?")
    quest_count = len(quest.findall(sent))
    return quest_count

In [28]:
def three_points(sent):
    three_points = re.compile(r"\.{3,}")
    three_points_count = len(three_points.findall(sent))
    return three_points_count

In [29]:
def determ_emotional(sent):
    emotion = re.compile(r"(\?{2,})|(!{2,})")
    emotion_counts = len(emotion.findall(sent))
    return emotion_counts 

In [30]:
def quant_sentence(sent):
    quant_sent = re.compile(r"(\.{1,})|(!{1,})|(\?{1,})")
    quant_sent_count = len(quant_sent.findall(sent))
    return quant_sent_count

In [36]:
X_qs = np.asarray(X.apply(quant_sentence))
X_de = np.asarray(X.apply(determ_emotional))
X_tp = np.asarray(X.apply(three_points))
X_q = np.asarray(X.apply(quest))
X_v = np.asarray(X.apply(voskl))
X_l = np.asarray(X.apply(len_of_sentence))

In [40]:
additional_features = [X_qs, X_de, X_tp, X_q, X_v, X_l]

In [42]:
reshaped_features = []
for feature in additional_features:
    reshaped_features.append(feature.reshape(feature.shape[0],1))
reshaped_features.append(X_13gram_pos_tfidf)

In [43]:
X_plus_eng = sp.sparse.hstack(reshaped_features)

In [44]:
score_eng = get_score_lgbm(X_plus_eng, y)

Дополнительные фичи ухудшили результат на 0.00012

Добавим извлечение синтаксиса

In [112]:
def get_deps(text):
    sentences = syntax_model.tokenize(text)
    deps = []
    for s in sentences:
        syntax_model.tag(s)
        syntax_model.parse(s)
        for word in s.words:
            deps.append(word.deprel)
    return " ".join(deps)

In [115]:
X_synt = X.apply(get_deps)

In [116]:
vec_synt = sklearn.feature_extraction.text.TfidfVectorizer(
    sublinear_tf=True)
X_synt_tf = vec_synt.fit_transform(X_synt)

In [117]:
X_plus_synt = sp.sparse.hstack([X_synt_tf, X_13gram_pos_tfidf])

In [119]:
score_synt = get_score_lgbm(X_plus_synt, y)

Добавление информации о синтаксисе ухудшило результат на 0.00016

## Score сабмитов на kaggle

Алгоритмы, которые использовались для морфологического и синтаксического анализа достаточно тяжелые, к тому же не дали положительного или сильно положительного результата при оценке на уменьшенной выборке, поэтому сабмитов с этим типом обработки данных нет. 

Score с kaggle: 
- простой CountVectorizer слов - 0.2456
- Tfidf слов - 0.2447
- Tfidf слов и 3-грамм - 0.2446
- Символьные n-граммы - 0.2479

Некоторые примечания:
- так как лемматизация "ломалась" из-за присутсвия некириллических символов, мы вычистили тексты от англоязычных слов. Однако это скорее плохое решение, т.к. среди таких слов могут оказываться названия фирм и брендов, что влияет на успешность объявления. Возможное решение: до нормализации текста вынести англоязычные слова в отдельное признаковое поле.
- синтаксис не нужен. 
- из всех наворотов результат принесло только определение последовательностей POS_тегов
- в задачах такого плана для создания успешного алгоритма необходимо работать со всеми данными, не только текстами объявлений (так, добавление зависимости между ценой товара и средней ценой по данной категории товаров дало небольшой прирост, но мы куда-то потеряли этот момент).